In [1]:
# !pip install opencv-python
# !pip install opencv-contrib-python

In [16]:
import cv2
from cv2 import dnn_superres
import moviepy
import moviepy.editor
import os

file_name = 'low_res_2'
ext = 'mp4'
scale = 4
input_video = f'./input/videos/{file_name}.{ext}'
output_root_path = f'./output/videos/{file_name}'
os.mkdir(output_root_path)


In [17]:
from moviepy.editor import AudioFileClip

video = moviepy.editor.VideoFileClip(input_video)
audio = video.audio
audio

In [4]:
# from tqdm.auto import tqdm

# # bicubic upsampling

# vs = cv2.VideoCapture(input_video)
# success, frame = vs.read()
# input_width = frame.shape[1] # 640
# input_height = frame.shape[0] # 352
# frame_count = 0
# total_frames = vs.get(cv2.CAP_PROP_FRAME_COUNT)
# frame_rate = vs.get(cv2.CAP_PROP_FPS)
# print(f'input res: {frame.shape}, total frames={total_frames}, frame rate={frame_rate}')
# first_part = output_video.split('.')[1]
# second_part = output_video.split('.')[2]
# output_path = first_part+'_'+'cubicx_'+scale+'.'+second_part
# out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'),
#                          frame_rate, (scale*input_width, scale*input_height))

# with tqdm(total=total_frames) as pbar:
#     while success:
#         upsampled_frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
#         out.write(upsampled_frame)

#         frame_count += 1
#         pbar.update(1)
#         pbar.set_description(f'frame no: {frame_count}')
#         success, frame = vs.read()
#     pbar.close()

# vs.release()
# out.release()
# print('done')

In [5]:
# import numpy as np
# import os
# from tqdm.auto import tqdm

# sr = dnn_superres.DnnSuperResImpl_create()
# fullModelName = 'LapSRN_x8.pb'
# first_part = output_video.split('.')[1]
# second_part = output_video.split('.')[2]
# output_path = '.'+first_part+'_'+fullModelName.replace('.pb', '')+'.'+second_part
# path = f"./input/models/{fullModelName}"
# modelName = fullModelName.split(os.path.sep)[-1].split("_")[0].lower()
# modelScale = fullModelName.split("_x")[-1]
# modelScale = int(modelScale[:modelScale.find(".")])

# vs = cv2.VideoCapture(input_video)
# success, frame = vs.read()
# input_width = frame.shape[1] # 640
# input_height = frame.shape[0] # 352
# frame_count = 0
# total_frames = vs.get(cv2.CAP_PROP_FRAME_COUNT)
# frame_rate = vs.get(cv2.CAP_PROP_FPS)
# print(f'input res: {frame.shape}, total frames={total_frames}, frame rate={frame_rate},\
#        scale={modelScale}, outpath={output_path}')

# sr.readModel(path)
# sr.setModel(modelName, modelScale)

# out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'),
#                          frame_rate, (modelScale*input_width, modelScale*input_height))

# with tqdm(total=total_frames) as pbar:
#     while success:
#         upsampled_frame = sr.upsample(frame)
#         out.write(upsampled_frame)

#         frame_count += 1
#         pbar.update(1)
#         pbar.set_description(f'frame no: {frame_count}')
#         success, frame = vs.read() 
#         if frame_count > 150:
#             break
#     pbar.close()

# vs.release()
# out.release()
# print('done')

In [6]:
# import numpy as np
# import os
# from tqdm.auto import tqdm
# from diffusers import LDMSuperResolutionPipeline, StableDiffusionUpscalePipeline
# import torch
# from PIL import Image

# fullModelName = 'stabilityai-stable-diffusion-x4-upscaler' # 'CompVis-ldm-super-resolution-4x-openimages'
# model_id = 'stabilityai/stable-diffusion-x4-upscaler'  # "CompVis/ldm-super-resolution-4x-openimages"
# first_part = output_video.split('.')[1]
# second_part = output_video.split('.')[2]
# output_path = '.'+first_part+'_'+fullModelName.replace('.pb', '')+'.'+second_part

# modelName = fullModelName  # .split(os.path.sep)[-1].split("_")[0].lower()
# modelScale = 4
# # pipeline = LDMSuperResolutionPipeline.from_pretrained(model_id)
# pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# pipeline = pipeline.to(device)

In [7]:
# vs = cv2.VideoCapture(input_video)
# success, frame = vs.read()
# input_width = frame.shape[1] # 640
# input_height = frame.shape[0] # 352
# frame_count = 0
# total_frames = vs.get(cv2.CAP_PROP_FRAME_COUNT)
# frame_rate = vs.get(cv2.CAP_PROP_FPS)
# print(f'input res: {frame.shape}, total frames={total_frames}, frame rate={frame_rate},\
#        scale={modelScale}, outpath={output_path}')

# out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'),
#                          frame_rate, (modelScale*input_width, modelScale*input_height))

# with tqdm(total=total_frames) as pbar:
#     while success:
#         if frame_count == 100:
#             srcImg = Image.fromarray(frame)

#             tgtImg = pipeline(srcImg, num_inference_steps=50, eta=1).images[0]
#             srcImg.show()
#             tgtImg.show()
#             break
#         # upsampled_frame = pipeline(Image.fromarray(frame), num_inference_steps=100, eta=1).images[0]
#         # out.write(np.array(upsampled_frame))
#         frame_count += 1
#         pbar.update(1)
#         pbar.set_description(f'frame no: {frame_count}')
#         success, frame = vs.read()
#         # if frame_count > 150:
#         #     break
#     pbar.close()

# vs.release()
# out.release()
# print('done')

In [48]:
list(map(lambda file: output_root_path+'/'+file, os.listdir(output_root_path)))

from pathlib import Path
paths = sorted(Path(output_root_path).iterdir(), key=os.path.getmtime)
files_sorted = list(map(str, sorted(Path(output_root_path).iterdir(), key=os.path.getmtime)))
files_sorted

['output/videos/low_res_2/low_res_2_BSRGAN_0.mp4',
 'output/videos/low_res_2/low_res_2_BSRGAN_1.mp4',
 'output/videos/low_res_2/low_res_2_BSRGAN_2.mp4',
 'output/videos/low_res_2/low_res_2_BSRGAN_3.mp4']

In [32]:
import numpy as np
from tqdm.auto import tqdm
import torch
from PIL import Image
import time
import sys
sys.path.append(os.path.abspath('/home/asutosh/Documents/ml_projects/BSRGAN'))
from models.network_rrdbnet import RRDBNet as net
from utils import utils_image as util

def get_output_file_path(file_name, model_name, file_part, parent_path, ext):
    file_prefix = f'{file_name}_{model_name}_{file_part}'
    output_path = f'{parent_path}/{file_prefix}.{ext}'
    return output_path

model_name = 'BSRGAN'
file_part = len(os.listdir(output_root_path))
output_part_path = get_output_file_path(file_name, model_name, file_part, output_root_path, ext)
number_of_frames_completed = -1

if file_part == 0:
    number_of_frames_completed = 0
else :
    file_parts = os.listdir(output_root_path)
    for file_part_ in file_parts:
        vs = cv2.VideoCapture(f'{output_root_path}/{file_part_}')
        number_of_frames_completed += vs.get(cv2.CAP_PROP_FRAME_COUNT)

modelScale = 4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = f'./input/models/{model_name}.pth'
time_limit = 6*60 # seconds
start_time = time.time()

torch.cuda.empty_cache()

model = net(in_nc=3, out_nc=3, nf=64, nb=23, gc=32, sf=modelScale)  # define network
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)
torch.cuda.empty_cache()

vs = cv2.VideoCapture(input_video)
vs.set(cv2.CAP_PROP_POS_FRAMES, number_of_frames_completed)
success, frame = vs.read()
input_width = frame.shape[1] # 640
input_height = frame.shape[0] # 352
frame_count = 0
total_frames = vs.get(cv2.CAP_PROP_FRAME_COUNT)
frame_rate = vs.get(cv2.CAP_PROP_FPS)
out_resolution = (modelScale*input_width, modelScale*input_height)

print(f'input res: {frame.shape}, total frames={total_frames}, frame rate={frame_rate}, scale={modelScale}, outpath={output_part_path}, completed_frames={number_of_frames_completed}')

codec = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_part_path, codec,
                         frame_rate, out_resolution)

with tqdm(total=total_frames) as pbar:
    pbar.update(number_of_frames_completed)
    frame_count = number_of_frames_completed
    while success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = util.uint2tensor4(frame).to(device)
        upsampled_frame = model(frame)
        upsampled_frame = util.tensor2uint(upsampled_frame)
        upsampled_frame = cv2.cvtColor(upsampled_frame, cv2.COLOR_RGB2BGR)
        
        out.write(upsampled_frame)
        frame_count += 1
        pbar.update(1)
        pbar.set_description(f'frame no: {frame_count}')
        success, frame = vs.read()
        if frame_count % 50:
            # check if run time > time_limit
            if time.time() - start_time > time_limit:
                break
            
    pbar.close()

vs.release()
out.release()
print(f'finished {frame_count} frames ({frame_count/total_frames*100}%)')

[3, 3, 64, 23, 32, 4]
input res: (240, 320, 3), total frames=913.0, frame rate=30.0, scale=4, outpath=./output/videos/low_res_2/low_res_2_BSRGAN_3.mp4, completed_frames=325.0


  0%|          | 0/913.0 [00:00<?, ?it/s]

finished 913.0 frames (100.0%)


In [49]:
from pathlib import Path

def concatenate_videos(new_video_path, codec, fps, resolution, videos):
    video = cv2.VideoWriter(new_video_path, codec, fps, resolution)

    for v in tqdm(videos):
        curr_v = cv2.VideoCapture(v)
        while curr_v.isOpened():
            r, frame = curr_v.read()
            if not r:
                break
            video.write(frame)

    video.release()
    print(f'video combined at {new_video_path}')

files_sorted = list(map(str, sorted(Path(output_root_path).iterdir(), key=os.path.getmtime)))
concat_path = get_output_file_path(file_name, model_name, 'all', output_root_path, ext)
concatenate_videos(concat_path, codec, frame_rate, out_resolution, files_sorted)

  0%|          | 0/4 [00:00<?, ?it/s]

video combined at ./output/videos/low_res_2/low_res_2_BSRGAN_all.mp4


In [50]:
video_out = moviepy.editor.VideoFileClip(concat_path)
audio = audio.volumex(2)
video_with_audio = video_out.set_audio(audio)
output_video_audio = get_output_file_path(file_name, model_name, 'final', output_root_path, ext)
video_with_audio.write_videofile(output_video_audio)

Moviepy - Building video ./output/videos/low_res_2/low_res_2_BSRGAN_final.mp4.
MoviePy - Writing audio in low_res_2_BSRGAN_finalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./output/videos/low_res_2/low_res_2_BSRGAN_final.mp4



Moviepy - Done !
Moviepy - video ready ./output/videos/low_res_2/low_res_2_BSRGAN_final.mp4
